Takes as input a .json review file and outputs 2 .json files with train and test data, such that all the users and items in test are present in train too. k controls how many rows are moved from train to test for each user (default=1). Takes a couple of minutes to run for 700k reviews.

In [50]:
# Imports 
import string
import pandas as pd
import numpy as np
import time
from scipy.stats.stats import pearsonr 
from sklearn.model_selection import train_test_split
import json

k=1 #take k rows from each user

# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns]
    return dataframe, dataframe.values

train_df, values = load_required_data('../reviews_Apps_for_Android_5.json', ["asin", "reviewerID", "overall"])

print "Data loaded"

#sort by reviewerID
train_df.sort_values(by=['reviewerID'],inplace=True)

print "Sorted"

last_user = 'undef' #first reviewerID
row_dicts = []
row_indices = []

def markRowForTestSet(index, row):
        row_dicts.append(row.to_dict()) # saving values for copying to train_df
        row_indices.append(index) # saving indices for deletion

#for each userid in train_df
for index, row in train_df.iterrows():
    if row['reviewerID'] != last_user: # first row for this user
        markRowForTestSet(index, row)
        n=k-1
    elif n>0:
        markRowForTestSet(index, row)
        n-=1        
    last_user = row['reviewerID']
    
print "Rows Marked"
    
test_df = pd.DataFrame.from_dict(row_dicts)

print "Train_df had",len(train_df.reviewerID.unique()),"unique reviewers and",len(train_df.asin.unique()),"unique items"

train_df.drop(row_indices, inplace=True)

print "train_df now has ",len(train_df),"reviews left"
print "test_df has",len(test_df.reviewerID.unique()),"unique reviewers and",len(test_df.asin.unique()),"unique items"
print "test_df now has ",len(test_df),"reviews"

#write to files
test_df.to_json("test.json",
           orient="records", lines=True)    

train_df.to_json("train.json",
           orient="records", lines=True)

print("Done!")

Data loaded
Sorted
Rows Marked
Train_df had 87271 unique reviewers and 13209 unique items
train_df now has  665666 reviews left
test_df has 87271 unique reviewers and 10061 unique items
test_df now has  87271 reviews
Done!
